In [ ]:
import pandas as pd
import dotenv
import os
from libraries.client_stashapp import get_stashapp_client
from libraries.StashDbClient import StashDbClient

dotenv.load_dotenv()

stash = get_stashapp_client()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

In [ ]:
galleries = stash.find_galleries()

In [ ]:
df_galleries = pd.DataFrame(galleries)

In [ ]:
# Filter galleries with a single file
df_single_file_galleries = df_galleries[df_galleries['files'].apply(lambda x: len(x) == 1)].copy()

# Extract basename of the single file
df_single_file_galleries.loc[:, 'file_basename'] = df_single_file_galleries['files'].apply(lambda x: os.path.basename(x[0]['basename']))

# Reset index for cleaner output
df_single_file_galleries = df_single_file_galleries.reset_index(drop=True)

# Select only the specified columns
df_single_file_galleries = df_single_file_galleries[['id', 'date', 'title', 'file_basename']]

# Parse date from file_basename and create a new column
df_single_file_galleries['parsed_date'] = df_single_file_galleries['file_basename'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]

df_single_file_galleries = df_single_file_galleries[(df_single_file_galleries['date'].isna()) & ((df_single_file_galleries['parsed_date'].notna()) & (df_single_file_galleries['parsed_date'] != "0001-01-01"))]


In [ ]:
for index, row in df_single_file_galleries.iterrows():
    stash.update_gallery({
        "id": row["id"],
        "date": row["parsed_date"]
    })